# Entrenamiento de modelos de clasificación

En este notebook se entrenan distintos modelos de clasificación supervisada con el objetivo
de predecir la enfermedad a partir de la combinación de síntomas. Se comparan modelos
lineales y no lineales para evaluar su rendimiento y seleccionar el más adecuado.


## Importación de librerías

In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import joblib


## Carga del dataset procesado

In [10]:
ruta = "../datos/procesados/enfermedades_sintomas_limpio.csv"
df = pd.read_csv(ruta)

df.shape

(189647, 378)

Se carga el dataset procesado generado en las fases anteriores, que contiene únicamente
datos limpios y preparados para el entrenamiento de modelos.

## Separación de variables

In [11]:
X = df.drop(columns=['diseases'])
y = df['diseases']

La variable objetivo corresponde a la enfermedad, mientras que el resto de variables
representan síntomas codificados de forma binaria.

## Codificación de la variable objetivo

In [12]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

y_encoded[:10]

array([531, 531, 531, 531, 531, 531, 531, 531, 531, 531])

Dado que la variable objetivo es categórica, se aplica una codificación mediante
LabelEncoder para convertir las etiquetas de texto en valores numéricos.


## División entrenamiento / test


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42
)



El conjunto de datos se divide en un 80% para entrenamiento y un 20% para test.
No se aplica estratificación en la división debido a que algunas enfermedades aparecen
únicamente una vez en el dataset, lo que impide garantizar una distribución estratificada
válida sin introducir errores.


## Modelo 1: Regresión logística

In [20]:
log_reg = LogisticRegression(
    max_iter=1000,
    n_jobs=-1
)

log_reg.fit(X_train, y_train)

y_pred_lr = log_reg.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)

acc_lr



0.834748220406011

El rendimiento obtenido indica que un modelo lineal es capaz de capturar eficazmente
la relación entre los síntomas y la enfermedad, estableciendo una referencia sólida
para la comparación con modelos más complejos.


## Modelo 2: Random Forest

In [22]:
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)

acc_rf


0.7899024518850514

El modelo Random Forest permite capturar relaciones no lineales entre los síntomas y
la enfermedad, siendo especialmente adecuado para conjuntos de datos con alta
dimensionalidad.


## Comparación de modelos

In [23]:
resultados = pd.DataFrame({
    "Modelo": ["Regresión Logística", "Random Forest"],
    "Accuracy": [acc_lr, acc_rf]
})

resultados



,Modelo,Accuracy
0,Regresión Logística,0.834748
1,Random Forest,0.789902


Los resultados obtenidos muestran que la regresión logística supera al modelo Random
Forest en términos de precisión. Este comportamiento puede explicarse por la naturaleza
binaria de las variables de entrada y la elevada dimensionalidad del problema, donde
modelos lineales tienden a ofrecer una mayor estabilidad.

En consecuencia, se selecciona la regresión logística como modelo principal para la fase
de evaluación, priorizando tanto el rendimiento como la interpretabilidad del modelo.


## Guardado de modelos

In [18]:
joblib.dump(log_reg, "../modelos/regresion_logistica.pkl")
joblib.dump(rf, "../modelos/random_forest.pkl")

['../modelos/random_forest.pkl']

Los modelos entrenados se guardan para su posterior uso en la aplicación y en la fase
de evaluación.


## Conclusiones del entrenamiento

Aunque el modelo Random Forest es capaz de modelar relaciones no lineales, su rendimiento
inferior puede deberse al elevado número de clases y a la limitada cantidad de muestras
por enfermedad, lo que dificulta la generalización de los árboles individuales.

En este escenario, la regresión logística se comporta como un modelo más estable,
demostrando que una mayor complejidad no siempre implica un mejor rendimiento.


En base a los resultados obtenidos, la regresión logística se selecciona como el modelo
principal para la fase de evaluación final, al presentar un mejor equilibrio entre
precisión, eficiencia computacional e interpretabilidad.
